In [18]:
# Define las columnas numéricas y categóricas
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = ['ocean_proximity']

# Define la transformación para las características categóricas (OneHotEncoder)
categorical_transformer = OneHotEncoder()

# Aplicar la transformación a las características categóricas
X_train_cat_encoded = categorical_transformer.fit_transform(X_train[categorical_features])

# Obtener los nombres de las columnas codificadas
encoded_column_names = categorical_transformer.get_feature_names_out(categorical_features)

# Crear un DataFrame con las características categóricas codificadas
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded.toarray(), columns=encoded_column_names)

# Restablecer los índices de los DataFrames antes de la concatenación
X_train_reset_index = X_train.drop(columns=categorical_features).reset_index(drop=True)
X_train_cat_encoded_df_reset_index = X_train_cat_encoded_df.reset_index(drop=True)

# Combinar las características numéricas originales con las características categóricas codificadas
X_train_combined = pd.concat([X_train_reset_index, X_train_cat_encoded_df_reset_index], axis=1)
print(X_train_combined.shape[0])

# Define la transformación para las características numéricas (StandardScaler)
numeric_transformer = StandardScaler()

# Aplicar la transformación numérica a las columnas resultantes de la codificación One-Hot
X_train_combined_scaled = numeric_transformer.fit_transform(X_train_combined)

# Convertir el resultado en un DataFrame
X_train_combined_scaled_df = pd.DataFrame(X_train_combined_scaled, columns=X_train_combined.columns)
print(X_train_combined_scaled_df.isna().any())

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
1606,-121.14,40.29,17.0,1944.0,394.0,384.0,172.0,1.6875
4186,-118.45,34.29,30.0,762.0,228.0,840.0,226.0,2.3375
